In [39]:
import pandas as pd
import numpy as np
import ast
import json

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [40]:
#initializing firestore
# 1111_test - document to remove
cred = credentials.Certificate('/Users/dzianis/github/findremote-firebase-adminsdk-p9cw7-633a39d4a9.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [25]:
df = pd.read_csv('../scrap/greenhouse/data/jobs_list.csv')

In [26]:
df.head()

,company,job_id,job_name,location,job_url,img_url,datetime,job_description,swe,ds,de,em,pm,dis,mr,tags,tags_short
0,GitHub,4177775,"Design Director – Education, Communities, and ...",Remote - US East,/github/jobs/4177775,logo/GitHub.jpg,2022-08-12 14:57:05.277160,"<div class="""" id=""content"">\n<p>GitHub is chan...",False,False,False,False,False,True,False,"['git', 'api', 'shell', 'express', 'github', '...","['git', 'api', 'shell', 'express', 'github']"
1,GitHub,3992208,"Design Director – Education, Communities, and ...",Remote - US,/github/jobs/3992208,logo/GitHub.jpg,2022-08-12 14:57:10.878391,"<div class="""" id=""content"">\n<p>GitHub is chan...",False,False,False,False,False,True,False,"['git', 'api', 'shell', 'express', 'github', '...","['git', 'api', 'shell', 'express', 'github']"
2,GitHub,3992209,"Design Director – Education, Communities, and ...",Remote - Canada,/github/jobs/3992209,logo/GitHub.jpg,2022-08-12 14:57:16.365304,"<div class="""" id=""content"">\n<p>GitHub is chan...",False,False,False,False,False,True,False,"['git', 'api', 'shell', 'express', 'github', '...","['git', 'api', 'shell', 'express', 'github']"
3,GitHub,4020252,"Senior Engineering Manager, Primer Brand & Bra...",Remote - Europe,/github/jobs/4020252,logo/GitHub.jpg,2022-08-12 14:57:22.055988,"<div class="""" id=""content"">\n<p>GitHub is look...",True,False,False,True,False,False,False,"['javascript', 'java', 'css', 'ruby', 'git', '...","['javascript', 'java', 'css', 'ruby', 'git']"
4,GitHub,4098600,Lead Interaction Designer,Remote - US,/github/jobs/4098600,logo/GitHub.jpg,2022-08-12 14:57:27.657470,"<div class="""" id=""content"">\n<p><span style=""f...",False,False,False,False,False,True,False,"['c#', 'git', 'azure', 'shell', 'express', 'se...","['c#', 'git', 'azure', 'shell', 'express']"


In [28]:
df['location_simp'] = df['location'].apply(lambda x: x if len(x)<=30 else 'Remote')

In [29]:
df.job_url.apply(lambda x: x.split('/')[1]).unique()

array(['github', 'gitlab', 'invision', 'blockchain', 'automatticcareers',
       'monzo', 'mozilla', 'autoscout24', 'zapiercareers'], dtype=object)

In [30]:
# sort by tag_id
df = df.sort_values(by='job_id')

# convert strings to lists
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x))
df['tags_short'] = df['tags_short'].apply(lambda x: ast.literal_eval(x))

In [31]:
columns_to_write = ['company', 'job_id', 'datetime', 'img_url', 'job_name', 'tags_short', 'location_simp', 'job_url', 'swe', 'ds', 'de', 'em', 'pm', 'dis', 'mr']
with open('json_listings.json', 'w') as outfile:
    outfile.write(df[columns_to_write].to_json(orient='records'))

In [50]:
# create content data
data_content = dict()

for company in df.company.unique():
    data_content[company] = dict()
    for i in df[df.company == company].index:
        job_id = str(df.loc[df.company == company, 'job_id'][i])
        datetime_ = df.loc[df.company == company, 'datetime'][i]
        job_url = 'https://boards.greenhouse.io' + df.loc[df.company == company, 'job_url'][i]
        job_name = df.loc[df.company == company, 'job_name'][i]
        img_url = df.loc[df.company == company, 'img_url'][i]
        company_name = df.loc[df.company == company, 'company'][i]
        location = df.loc[df.company == company, 'location'][i]
        location_simp = df.loc[df.company == company, 'location_simp'][i]
        tags = df.loc[df.company == company, 'tags'][i]
        tags_short = df.loc[df.company == company, 'tags_short'][i]
        
        swe = df.loc[df.company == company, 'swe'][i]
        ds = df.loc[df.company == company, 'ds'][i]
        de = df.loc[df.company == company, 'de'][i]
        em = df.loc[df.company == company, 'em'][i]
        pm = df.loc[df.company == company, 'pm'][i]
        dis = df.loc[df.company == company, 'dis'][i]
        mr = df.loc[df.company == company, 'mr'][i]
        
        job_description = df.loc[df.company == company, 'job_description'][i]
        job_description = job_description.replace('h5', 'h6')
        job_description = job_description.replace('h4', 'h6')
        job_description = job_description.replace('h3', 'h5')
        job_description = job_description.replace('h2', 'h4')
        job_description = job_description.replace('h1', 'h4')
        
        
        # write to Fire Store (Content)
        doc_ref = db.collection(u'listings').document(job_id)
        doc_ref.set({
            u'job_id': job_id,
            u'datetime': datetime_,
            u'job_url': job_url,
            u'job_name': job_name,
            u'img_url': img_url,
            u'company_name': company_name,
            u'location': location,
            u'location_simp': location_simp,
            u'tags': tags,
            u'tags_short': tags_short,
            u'job_description': job_description,
            u'swe': bool(swe),
            u'ds': bool(ds),
            u'de': bool(de),
            u'em': bool(em),
            u'pm': bool(pm),
            u'dis': bool(dis),
            u'mr': bool(mr)
        })

        
        # write to Fire Store (Short version of listings)
        doc_ref = db.collection(u'listings_light').document(job_id)
        doc_ref.set({
            u'job_id': job_id,
            u'datetime': datetime_,
            u'job_name': job_name,
            u'img_url': img_url,
            u'company_name': company_name,
            u'location_simp': location_simp,
            u'tags_short': tags_short,
            u'swe': bool(swe),
            u'ds': bool(ds),
            u'de': bool(de),
            u'em': bool(em),
            u'pm': bool(pm),
            u'dis': bool(dis),
            u'mr': bool(mr)
        })
        
        
        # write to json
        data_content[company][int(job_id)] = {
            'company_name': company_name,
            'job_name': job_name,
            'datetime': datetime_,
            'job_url': job_url,
            'img_url': img_url,
            'location': location,
            'tags': tags,
            'job_description': job_description
        }

with open('json_content.json', 'w') as outfile:
    json.dump(data_content, outfile)

In [33]:
from datetime import datetime

In [34]:
(datetime.utcnow() - pd.to_datetime(datetime_)) / pd.Timedelta('1 hour')

3.3870097827777776

In [35]:
df.head()

,company,job_id,job_name,location,job_url,img_url,datetime,job_description,swe,ds,de,em,pm,dis,mr,tags,tags_short,location_simp
335,Blockchain,1238688,Front End Engineer,"London, San Francisco, Miami, Remote",/blockchain/jobs/1238688,https://storage.googleapis.com/findremote/bloc...,2022-08-12 15:28:37.342973,"<div class="""" id=""content"">\n<p><strong>Blockc...",True,False,False,False,False,False,False,"[javascript, java, git, aws, api, performance,...","[javascript, java, git, aws, api]",Remote
340,Blockchain,1271504,Android Engineer,"London, San Francisco, Miami, Remote",/blockchain/jobs/1271504,https://storage.googleapis.com/findremote/bloc...,2022-08-12 15:29:08.315855,"<div class="""" id=""content"">\n<p><strong>Blockc...",True,False,False,False,False,False,False,"[java, android, git, aws, api, performance, un...","[java, android, git, aws, api]",Remote
341,Blockchain,1271531,iOS Engineer,"London, San Francisco, Miami, Remote",/blockchain/jobs/1271531,https://storage.googleapis.com/findremote/bloc...,2022-08-12 15:29:13.786000,"<div class="""" id=""content"">\n<p><strong>Blockc...",True,False,False,False,False,False,False,"[swift, objective-c, git, aws, api, performanc...","[swift, objective-c, git, aws, api]",Remote
339,Blockchain,1277974,"Senior Software Engineer, Low Latency","London, San Francisco, Miami, Remote",/blockchain/jobs/1277974,https://storage.googleapis.com/findremote/bloc...,2022-08-12 15:29:02.823308,"<div class="""" id=""content"">\n<p><strong>Blockc...",True,False,False,False,False,False,False,"[java, c++, sql, database, postgresql, git, aw...","[java, c++, sql, database, postgresql]",Remote
327,Blockchain,1277981,"Senior Software Engineer, Platform","London, San Francisco, Miami, Remote",/blockchain/jobs/1277981,https://storage.googleapis.com/findremote/bloc...,2022-08-12 15:27:53.522607,"<div class="""" id=""content"">\n<p><strong>Blockc...",True,False,False,False,False,False,False,"[java, sql, database, postgresql, git, aws, al...","[java, sql, database, postgresql, git]",Remote
